In [ ]:
#!/usr/bin/env julia
"""
Calculate road length distribution from a graph JSON file.

Usage:
    julia road_lengths.jl input.json output.csv
"""

using JSON
using DataFrames
using CSV

# Define ENU struct for coordinates
struct ENU
    east::Float64
    north::Float64
    up::Float64
end

# Distance function (provided)
function distance_points(point1::ENU, point2::ENU)
    east1 = point1.east
    east2 = point2.east
    north1 = point1.north
    north2 = point2.north
    return sqrt((east2 - east1)^2 + (north2 - north1)^2)
end

"""
Load graph from JSON file and extract nodes and edges.
"""
function load_graph(filepath::String)
    data = JSON.parsefile(filepath)
    level1 = data["level1"]
    
    # Parse nodes into Dict{Int, ENU}
    nodes = Dict{Int, ENU}()
    for (node_id, coords) in level1["nodes"]
        nodes[Int(node_id)] = ENU(
            Float64(coords["east"]),
            Float64(coords["north"]),
            Float64(get(coords, "up", 0.0))
        )
    end
    
    # Parse edges
    edges = [(Int(e[1]), Int(e[2])) for e in level1["edges"]]
    
    return nodes, edges
end

"""
Calculate lengths of all edges (roads) in the graph.
Returns a DataFrame with a single column 'length'.
"""
function calculate_road_lengths(filepath::String)
    nodes, edges = load_graph(filepath)
    
    lengths = Float64[]
    
    for (n1, n2) in edges
        point1 = nodes[n1]
        point2 = nodes[n2]
        len = distance_points(point1, point2)
        push!(lengths, len)
    end
    
    return DataFrame(length = lengths)
end

"""
Main function.
"""
function main()
    if length(ARGS) < 2
        println("Usage: julia road_lengths.jl input.json output.csv")
        return
    end
    
    input_path = ARGS[1]
    output_path = ARGS[2]
    
    println("Loading graph from $input_path...")
    df = calculate_road_lengths(input_path)
    
    println("Calculated $(nrow(df)) road lengths")
    println("  Min:    $(round(minimum(df.length), digits=2)) m")
    println("  Max:    $(round(maximum(df.length), digits=2)) m")
    println("  Mean:   $(round(mean(df.length), digits=2)) m")
    println("  Median: $(round(median(df.length), digits=2)) m")
    
    CSV.write(output_path, df)
    println("Saved to $output_path")
end

if abspath(PROGRAM_FILE) == @__FILE__
    using Statistics
    main()
end